In [1]:
%reset -f

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The block below identifies ghcnd stations with at least some data in the 1974-2013 range, record completeness > 0%, and distance to nearest CLIGEN station up to 100000 m. Only a metadata file is created, and the records are not downloaded in this step. A file is read containing lat/lon coordinates of the U.S. CLIGEN network. Run time is ~27 min.

In [8]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime as dt
from datetime import timedelta
import os
import scipy as scipy
import statsmodels.api as sm
from google.colab import data_table
data_table.enable_dataframe_formatter()
import csv
import geopandas as gpd
from geopy.distance import geodesic


stationIDLINK = 'https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt'
metadataLINK = 'https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-inventory.txt'
dataLINK = 'https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/access'
coordFILE = '/content/drive/My Drive/Colab Notebooks/CLIGEN/US_CLIGEN_Coords.txt'
keepFILE = '/content/drive/My Drive/Colab Notebooks/TeamProjects/Keepers_John.csv'

max_dist = 100000.0 #meters
percent_complete = 0.0 #%

dates = []
start_date = dt(1974, 1, 1, 0, 0)
end_date = dt(2013, 12, 31, 0, 0)
date = start_date
while date <= end_date:
  dates.append(date)
  date = date + timedelta(days=1)

stations, longitudes, latitudes = [], [], []
with open(coordFILE) as f:
  next(f)
  for line in f:
    row = line.strip('\n').split(',')
    stations.append(row[0])
    longitudes.append(row[1])
    latitudes.append(row[2])

cli_df = pd.DataFrame(data=zip(stations, longitudes, latitudes), columns=['stationID', 'long', 'lat'])

url = stationIDLINK
req = requests.get(url)
text = req.text

stations, longitudes, latitudes = [], [], []
lines = (line for line in text.splitlines())
for line in lines:
  row = line.split()
  stationID = row[0]
  if stationID[:2] == 'US':
    stations.append(row[0])
    latitudes.append(row[1])
    longitudes.append(row[2])

gnd_df = pd.DataFrame(data=zip(stations, longitudes, latitudes), columns=['stationID', 'long', 'lat'])

#EPSG:4326 geodetic coordinates -> 'EPSG:3857' meters
cli_gdf = gpd.GeoDataFrame(
  cli_df, geometry=gpd.points_from_xy(cli_df['long'], cli_df['lat']), crs='EPSG:3857'
)

gnd_gdf = gpd.GeoDataFrame(
  gnd_df, geometry=gpd.points_from_xy(gnd_df['long'], gnd_df['lat']), crs='EPSG:3857'
)

join_gdf = gpd.sjoin_nearest(cli_gdf, gnd_gdf, how='left', distance_col='dist')
join_gdf.to_crs('EPSG:3857')

join_df = pd.DataFrame(join_gdf.drop(columns=['geometry', 'index_right']))
join_df['dist_m'] = join_df.apply(lambda x: geodesic((x['lat_left'], x['long_left']), (x['lat_right'], x['long_right'])).meters, axis=1)

keepers_one_df = join_df[join_df['dist_m'] < max_dist].reset_index()

keepers_step_one = keepers_one_df['stationID_right'].values

url = metadataLINK
req = requests.get(url)
text = req.text

keepers_step_two = []
lines = (line for line in text.splitlines())
for line in lines:
  row = line.split()
  if 'PRCP' in row and row[0] in keepers_step_one:
    if int(row[4]) <= 1974 and int(row[5]) >= 2013:
      keepers_step_two.append(row[0])

#No html address or too much missing data
bad = []
for keeper in keepers_step_two:

  try:
    ct = 0
    url = dataLINK + '/' + keeper + '.csv'
    req = requests.get(url)
    text = req.text
    if not '404 Not Found' in text:
      ct = 0
      lines = [line for line in text.splitlines()]
      save_lines = []
      hdrs = lines[0].split(',')
      prcp_i = hdrs.index('"PRCP"')
      date_i = hdrs.index('"DATE"')
      for line in lines[1:]:
        row = line.split('","')
        name_no_comma = row[5].replace(',', '')
        line = line.replace(row[5], name_no_comma)
        line = line.replace('","', ',')
        row = line.split(',')
        date = dt.strptime(row[date_i].strip('"'), '%Y-%m-%d')
        prcp = row[prcp_i].strip('"')
        if date.year >= 1974 and date.year <= 2013:
          if prcp != '' and not any([s in prcp for s in ['P', 'T', 'H', '9999']]):
            prcp = float(prcp)
            ct += 1

      error_per = float(ct)/float(len(dates))*100
      keepers_one_df.loc[keepers_one_df['stationID_right'] == keeper, 'complete_per'] = error_per
      if float(ct)/float(len(dates))*100. < percent_complete:
        pass

      if float(ct)/float(len(dates))*100. < percent_complete:
        print(str(float(ct)/float(len(dates))*100))
        bad.append(keeper)

    else:
      bad.append(keeper)

  except requests.exceptions.Timeout:
    bad.append(keeper)


keepers_step_three = [k for k in keepers_step_two if k not in bad]

keepers_df = keepers_one_df.loc[keepers_one_df['stationID_right'].isin(keepers_step_three)].reset_index()

keepers_df.to_csv(keepFILE)



The block below downloads the identified records. Run time is ~6 hr.

In [ ]:
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd
import requests
import os

keepFILE = '/content/drive/My Drive/Colab Notebooks/TeamProjects/Keepers_John.csv'
dataLINK = 'https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/access'
outFOLDER = '/content/drive/My Drive/Colab Notebooks/TeamProjects/Output/GHCN_Dataframes_for_Keepers_John'
keepers_df = pd.read_csv(keepFILE)
keepers = keepers_df['stationID_right'].values

dates = []
start_date = dt(1974, 1, 1, 0, 0)
end_date = dt(2013, 12, 31, 0, 0)
date = start_date
while date <= end_date:
  dates.append(date)
  date = date + timedelta(days=1)

i = 0
done = False
while done == False:

  keeper = keepers[i]
  print(keeper)
  df = pd.DataFrame()
  df.index.name = 'index'
  df['year'] = [date.year for date in dates]
  df['month'] = [date.month for date in dates]
  df['day'] = [date.day for date in dates]
  df[keeper] = [0.0 for date in dates]

  outFILE = os.path.join(outFOLDER, keeper + '.csv')
  url = dataLINK + '/' + keeper + '.csv'
  req = requests.get(url)
  text = req.text
  if not '404 Not Found' in text:

    lines = [line for line in text.splitlines()]
    hdrs = lines[0].split(',')
    prcp_col_i = hdrs.index('"PRCP"')
    date_col_i = hdrs.index('"DATE"')

    for line in lines[1:]:
      row = line.split('","')
      name_no_comma = row[5].replace(',', '')
      line = line.replace(row[5], name_no_comma)
      line = line.replace('","', ',')
      row = line.split(',')
      lon = row[3]
      lat = row[2]
      date = dt.strptime(row[date_col_i].strip('"'), '%Y-%m-%d')
      prcp = row[prcp_col_i].strip('"')

      if date.year >= 1974 and date.year <= 2013:

        if prcp != '' and not any([s in prcp for s in ['P', 'T', 'H', '9999']]):
          prcp = float(prcp)/10.0
          df.loc[(df['year'] == date.year) & (df['month']==date.month) & (df['day']==date.day), keeper] = prcp

  i += 1

  if i == len(keepers):
    done = True

  df.to_csv(outFILE)


